In [ ]:
# import modules and define functions
from pyscf import gto, scf, dft
import pandas as pd
import numpy as np
import os
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1" 

# Koopman's theorem

In [ ]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = 'def2-qzvpp'
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)

In [ ]:
# SCF calculations
# HF
h2_0_1_uhf = scf.UHF(h2_0_1).run()

In [ ]:
# Koopman's IP for H2
mo_occ = h2_0_1_uhf.mo_occ
mo_energy = h2_0_1_uhf.mo_energy
e_homo_h2 = mo_energy[mo_occ>0][-1]
print('HOMO energy of H2 at HF level is %.4f Eh' %(e_homo_h2))

In [ ]:
# calculate IP for H2
h2_0_1_uhf = scf.UHF(gto.Mole().build(atom=atom,basis=basis,charge=0,spin=0)).run()
h2_1_2_uhf = scf.UHF(gto.Mole().build(atom=atom,basis=basis,charge=1,spin=1)).run()
s2_h2 = h2_1_2_uhf.spin_square()[0]
IP_H2 = (h2_0_1_uhf.e_tot - h2_1_2_uhf.e_tot)
print('Ionization potential calculated from e_diff between H2 and H2+ is %.4f Eh' %(IP_H2))

In [ ]:
# calculate MO and IP for CO and N2
# CO
co_0_1 = gto.Mole().build(atom='C 0 0 0; O 0 0 1.13',basis=basis,charge=0,spin=0)
co_1_2 = gto.Mole().build(atom='C 0 0 0; O 0 0 1.13',basis=basis,charge=1,spin=1)
co_0_1_uhf = scf.UHF(co_0_1).run()
co_1_2_uhf = scf.UHF(co_1_2).run()
s2_co = co_1_2_uhf.spin_square()[0]
mo_occ = co_0_1_uhf.mo_occ
mo_energy = co_0_1_uhf.mo_energy
e_homo_co = mo_energy[mo_occ>0][-1]
IP_CO = (co_0_1_uhf.e_tot - co_1_2_uhf.e_tot)
# N2
# calculate MO and IP for CO and N2
n2_0_1 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=0,spin=0)
n2_1_2 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=1,spin=1)
n2_0_1_uhf = scf.UHF(n2_0_1).run()
n2_1_2_uhf = scf.UHF(n2_1_2).run()
s2_n2 = n2_1_2_uhf.spin_square()[0]
mo_occ = n2_0_1_uhf.mo_occ
mo_energy = n2_0_1_uhf.mo_energy
e_homo_n2 = mo_energy[mo_occ>0][-1]
IP_N2 = n2_0_1_uhf.e_tot - n2_1_2_uhf.e_tot

In [ ]:
data = [['H\u2082', '0.584', e_homo_h2, IP_H2, s2_h2],
        ['CO','0.510', e_homo_co, IP_CO, s2_co],
        ['N\u2082','0.573', e_homo_n2, IP_N2, s2_n2]]
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(pd.DataFrame(data, columns=['Molecules', 'Expt', 'Koopman', 'IP', '<S^2>']))

# Brillouin theorem

In [ ]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = '6-31g'
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)
h2_0_1_rhf = scf.RHF(h2_0_1).run()

In [ ]:
# get the mo_coeffs and the 2e and fock matrix elements
mo_coeff = h2_0_1_rhf.mo_coeff
mo_occ = h2_0_1_rhf.mo_occ
h = h2_0_1_rhf.get_hcore()
veff = h2_0_1_rhf.get_veff()
f = h2_0_1_rhf.get_fock()

In [ ]:
# get H and G
F = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, f, mo_coeff])

In [ ]:
colnames = ['\u03c8' + '%s' %(i) for i in range(F.shape[0])]
pd.set_option('display.chop_threshold', 0.000001)
pd.set_option('display.float_format', lambda x: '%.6f' % x)
df = pd.DataFrame(F, columns=colnames, index=colnames)
print(df)
